In [34]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf
import os

dir_path = 'CS341-repo/Data/'
df = pd.read_pickle(dir_path+'df_hourly_poloniex.pickle')
df = df.dropna()

df.head()

,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange,USDT_ETH_high,USDT_ETH_low,...,BTC_LTC_weighted_mean,BTC_LTC_pctChange,BTC_XRP_high,BTC_XRP_low,BTC_XRP_close,BTC_XRP_open,BTC_XRP_volume,BTC_XRP_quoteVolume,BTC_XRP_weighted_mean,BTC_XRP_pctChange
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 12:00:00,432.5000,432.50,432.500000,432.50000,40.041239,0.092581,432.500000,2.220446e-16,0.959136,0.959136,...,0.008063,-0.002293,0.000014,0.000014,0.000014,0.000014,0.033605,2408.822942,0.000014,-0.002859
2016-01-02 13:00:00,432.5000,432.50,432.500000,432.50000,0.000000,0.000000,432.986941,1.125876e-03,0.959136,0.959136,...,0.008060,-0.000333,0.000014,0.000014,0.000014,0.000014,0.000000,0.000000,0.000014,0.004704
2016-01-02 14:00:00,437.3635,432.48,433.336667,433.52799,359.269753,0.828819,433.473883,1.124610e-03,0.959136,0.957000,...,0.008073,0.001623,0.000014,0.000014,0.000014,0.000014,1.141981,81071.098773,0.000014,0.004682
2016-01-02 15:00:00,432.4800,432.48,432.480000,432.48000,60.859598,0.140722,432.480000,-2.292832e-03,0.957000,0.957000,...,0.008089,0.002002,0.000014,0.000014,0.000014,0.000014,2.120423,150622.792769,0.000014,-0.000492
2016-01-02 16:00:00,432.4800,432.48,432.480000,432.48000,0.000000,0.000000,432.350000,-3.005919e-04,0.957000,0.957000,...,0.008079,-0.001224,0.000014,0.000014,0.000014,0.000014,0.491516,35178.793196,0.000014,-0.007526


In [2]:
scaler_features = preprocessing.MinMaxScaler(feature_range=(0.1, 1))

In [24]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len, test_size=-1):
    # prepare one-hot labels
    labels = df['USDT_BTC_pctChange'].as_matrix().reshape([-1,1])
    labels = np.concatenate([(labels > 3e-3)*1, ((3e-3 > labels)&(labels > -3e-3))*1, (labels < -3e-3)*1],1)
    labels = labels[seq_len-1:] # so labels and data has same length
    
    feature_set = [x for x in range(56)] #[0,1,2,3,4,6,7]
    
    data_raw = df.as_matrix() # convert to numpy array
    # fit scaler
    data_raw = scaler_features.fit_transform(data_raw[:, feature_set])
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len, :])
    
    data = np.array(data)
    
    if test_size == -1: # split the old way
        n_train_valid_pairs = 3
        each_train_set_size_pct = 25
        each_valid_set_size_pct = 5

        each_train_set_size = round(each_train_set_size_pct/100*data.shape[0])
        each_valid_set_size = round(each_valid_set_size_pct/100*data.shape[0])

        x_train_sets = []
        y_train_sets = []
        x_valid_sets = []
        y_valid_sets = []
        used = 0

        for i in range(n_train_valid_pairs):
            x_train_sets.append(data[used : used + each_train_set_size,:-1,:]) # cannot see last day, which we aim to predict
            y_train_sets.append(labels[used : used + each_train_set_size, :])
            used += each_train_set_size

            x_valid_sets.append(data[used : used + each_valid_set_size,:-1,:])
            y_valid_sets.append(labels[used : used + each_valid_set_size, :])
            used += each_valid_set_size

        x_test = data[used : , :-1, :]
        y_test = labels[used : , :]

        x_train = np.concatenate(x_train_sets, axis=0)
        y_train = np.concatenate(y_train_sets, axis=0)
        x_valid = np.concatenate(x_valid_sets, axis=0)
        y_valid = np.concatenate(y_valid_sets, axis=0)
    
    else:
        x_test = data[-test_size : , :-1, :]
        y_test = labels[-test_size : , :]
        
        valid_start = data.shape[0] - test_size - int(test_size/2)
        x_valid = data[valid_start:-test_size, :-1, :]
        y_valid = labels[valid_start:-test_size, :]
        
        x_train = data[:valid_start, :-1, :]
        y_train = labels[:valid_start, :]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [25]:
# create train, test data
seq_len = 10 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df, seq_len, test_size=1996)
# y_train = y_train.reshape([-1,1])
# y_valid = y_valid.reshape([-1,1])
# y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (16981, 9, 56)
y_train.shape =  (16981, 3)
x_valid.shape =  (998, 9, 56)
y_valid.shape =  (998, 3)
x_test.shape =  (1996, 9, 56)
y_test.shape =  (1996, 3)


In [5]:
def f1_score_single(y_true, y_pred):
    TP = y_true.dot(y_pred) # zero or one
    FP = np.sum(y_pred > y_true) # sum over all k classes, zero or one
    FN = np.sum(y_pred < y_true) # sum over all k classes, zero or one
    
    if TP == 0: return 0.
    p = 1. * TP / (TP + FP)
    r = 1. * TP / (TP + FN)
    return 2 * p * r / (p + r)
    
def F1(y_true, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

## baseline

In [26]:
y_pred = np.roll(y_valid,1, axis=0)
print('baseline dev_F1=',F1(y_valid[1:], y_pred[1:]))
y_pred = np.roll(y_test,1, axis=0)
y_pred[0] = y_valid[-1] # be careful here
print('baseline test_F1=',F1(y_test, y_pred))

baseline dev_F1= 0.39618856569709127
baseline test_F1= 0.3712424849699399


In [27]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

x_1000_train, y_1000_train = get_next_batch(1000) # special batch of 1000 records in training set

In [57]:
class LSTM_Model(object):
    def __init__(self, seq_len):
        # parameters
        self.n_steps = seq_len-1 
        self.n_inputs = x_train.shape[-1]
        self.n_neurons = 100  # cell.state_size
        self.n_bins = 3 # be careful if you want to change this
        self.n_layers = 2
        self.batch_size = 100
        self.n_epochs = 0 # 0 means to train indefinitely
        self.train_set_size = x_train.shape[0]
        self.test_set_size = x_test.shape[0]
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.max_gradient_norm = 5
        with tf.variable_scope("LSTM_Model", initializer=tf.contrib.layers.xavier_initializer()):
            self.X = tf.placeholder(tf.float32, [None, self.n_steps, self.n_inputs])
            self.y = tf.placeholder(tf.float32, [None, self.n_bins])

            layers = [tf.contrib.rnn.LSTMCell(num_units=self.n_neurons, \
                                              initializer=tf.contrib.layers.xavier_initializer(), \
                                              activation=tf.nn.elu)
                     for layer in range(self.n_layers)]
            
#             layers = [tf.contrib.rnn.BasicRNNCell(num_units=self.n_neurons, \
#                                               activation=tf.nn.elu)
#                      for layer in range(self.n_layers)]
            
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

            outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, self.X, dtype=tf.float32)
            outputs = tf.nn.dropout(outputs, self.keep_prob) # dropout
            # 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
        
#             directly output
#             logits = outputs[:,self.n_steps-1,:] # keep only last output of sequence
            
            # ======== attn layer ===================
#             sim_mat = tf.matmul(outputs, tf.transpose(outputs, perm=[0,2,1]))
#             attn_dist = tf.nn.softmax(sim_mat, 2)
#             # (batchsize, n_steps, n_steps)
#             attn_outputs = tf.matmul(attn_dist, outputs)
#             # (batchsize, n_steps, n_bins)
            # ========================================
            
            stacked_outputs = tf.reshape(outputs, [-1, self.n_neurons]) 
            stacked_outputs = tf.layers.dense(stacked_outputs, self.n_bins)
            final_outputs = tf.reshape(stacked_outputs, [-1, self.n_steps, self.n_bins])
            
            self.final_logits = final_outputs[:, -1, :] # last timestep
            # (batchsize, n_bins), this is logits, not probs!!
            self.final_logits = tf.nn.dropout(self.final_logits, self.keep_prob) # dropout
            
            self.indices = tf.argmax(self.final_logits, axis=-1) # (batchsize, 1)
            self.preds = tf.one_hot(self.indices, depth=self.n_bins)
            
            self.each_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.final_logits, labels=self.y)
            self.loss = tf.reduce_mean(self.each_loss) 

            params = tf.trainable_variables()
            gradients = tf.gradients(self.loss, params)
            self.gradient_norm = tf.global_norm(gradients)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
            clipped_norm = tf.global_norm(clipped_gradients)
            self.param_norm = tf.global_norm(params)
            self.learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate_placeholder) 
            # training_op = optimizer.minimize(loss)
            self.training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

            # initialize parameters
#             sess = tf.Session()
            self.global_step = tf.Variable(0, name='global_step', trainable=False)
            self.saver = tf.train.Saver(max_to_keep=2)
            self.bestmodel_saver = tf.train.Saver(max_to_keep=2)
    
    def train(self, session, experiment_name, keep_prob_val):
        
        bestmodel_dir = experiment_name+'/best_ckpt'
        bestmodel_ckpt_path = bestmodel_dir+'/best.ckpt'
        best_valid_f1 = None
        # Make bestmodel dir if necessary
        if not os.path.exists(bestmodel_dir):
            os.makedirs(bestmodel_dir)
        
        ckpt = tf.train.get_checkpoint_state(experiment_name)
        v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
        if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
            self.saver.restore(sess, ckpt.model_checkpoint_path)
            iteration = self.global_step.eval(session=sess) # get last global_step
            print("Start from iteration:", iteration)
            lr = 1e-3
        else:
            print('There is not saved parameters. Creating model with fresh parameters.')
            sess.run(tf.global_variables_initializer())
            iteration = 0
            lr = 1e-2 # should fix this...

        old_loss = 1000
        
        while self.n_epochs == 0 or iteration*self.batch_size/self.train_set_size < self.n_epochs:
            iteration = iteration + 1
            x_batch, y_batch = get_next_batch(self.batch_size) # fetch the next training batch 

            # train on this batch
            sess.run(self.training_op, feed_dict={self.X: x_batch, self.y: y_batch, self.learning_rate_placeholder:lr, \
                                                 self.keep_prob: keep_prob_val})

            if iteration % 50 == 0:
                y_1000_train_pred, loss_val, param_norm_val, grad_norm_val = \
                    sess.run([self.preds, self.loss, self.param_norm, self.gradient_norm],\
                            feed_dict={self.X: x_1000_train, self.y:y_1000_train, \
                                        self.learning_rate_placeholder:lr, self.keep_prob: keep_prob_val})
                    
                if loss_val > old_loss * 1.2:
                    lr /= 2
                old_loss = loss_val

                y_valid_pred = sess.run(self.preds, feed_dict={self.X: x_valid, self.keep_prob: keep_prob_val})
                
                valid_f1 = F1(y_valid, y_valid_pred)
                print('%.2f epochs, iter %d: train_loss = %.9f, param_norm = %.3f, grad_norm = %.3f, train_F1/valid_F1 = %.6f/%.6f' \
                      %(iteration*self.batch_size/self.train_set_size, iteration, loss_val, param_norm_val, grad_norm_val, \
                        F1(y_1000_train, y_1000_train_pred), \
                        valid_f1))

                if best_valid_f1 is None or valid_f1 > best_valid_f1:
                    best_valid_f1 = valid_f1
                    print("======New best valid F1. Saving to %s..." % bestmodel_ckpt_path)
                    self.bestmodel_saver.save(sess, bestmodel_ckpt_path, global_step=self.global_step)
                
            if iteration % 100 == 0:
                self.global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
                save_path = self.saver.save(sess, "./"+experiment_name+"/model.ckpt", global_step=self.global_step)
                print('Saved parameters to %s' % save_path)

## run experiment

In [58]:
experiment_name="all_features_10_seq_100hidden_2layer_drop08_longer"

In [59]:
tf.reset_default_graph()
lstm_model = LSTM_Model(seq_len=seq_len)
sess = tf.Session()

In [60]:
lstm_model.train(session=sess, experiment_name=experiment_name, keep_prob_val=0.8)
# IMPORTANT:
# when you think F1 is not going to improve anymore, wait another 10 epochs. 
# if you see any better iteration that has not appeared before, keep waiting.

There is not saved parameters. Creating model with fresh parameters.
0.29 epochs, iter 50: train_loss = 1.030339956, param_norm = 22.309, grad_norm = 0.089, train_F1/valid_F1 = 0.520000/0.200401
======New best valid F1. Saving to all_features_10_seq_100hidden_2layer_drop08_longer/best_ckpt/best.ckpt...
0.59 epochs, iter 100: train_loss = 1.007503271, param_norm = 22.312, grad_norm = 0.044, train_F1/valid_F1 = 0.517000/0.417836
======New best valid F1. Saving to all_features_10_seq_100hidden_2layer_drop08_longer/best_ckpt/best.ckpt...
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-100
0.88 epochs, iter 150: train_loss = 1.053766727, param_norm = 22.860, grad_norm = 0.179, train_F1/valid_F1 = 0.481000/0.279559
1.18 epochs, iter 200: train_loss = 1.012766480, param_norm = 24.990, grad_norm = 0.063, train_F1/valid_F1 = 0.520000/0.364729
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-200
1.47 epochs, iter 250: train_l

13.54 epochs, iter 2300: train_loss = 0.971173823, param_norm = 67.516, grad_norm = 0.073, train_F1/valid_F1 = 0.522000/0.265531
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-2300
13.84 epochs, iter 2350: train_loss = 0.960855663, param_norm = 68.029, grad_norm = 0.099, train_F1/valid_F1 = 0.548000/0.371743
14.13 epochs, iter 2400: train_loss = 0.969024658, param_norm = 68.511, grad_norm = 0.057, train_F1/valid_F1 = 0.532000/0.245491
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-2400
14.43 epochs, iter 2450: train_loss = 0.997358859, param_norm = 69.126, grad_norm = 0.253, train_F1/valid_F1 = 0.541000/0.415832
14.72 epochs, iter 2500: train_loss = 0.967388093, param_norm = 69.901, grad_norm = 0.081, train_F1/valid_F1 = 0.545000/0.258517
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-2500
15.02 epochs, iter 2550: train_loss = 0.968555093, param_norm = 70.154, grad_norm = 0.13

27.38 epochs, iter 4650: train_loss = 0.966429949, param_norm = 89.603, grad_norm = 0.340, train_F1/valid_F1 = 0.547000/0.398798
27.68 epochs, iter 4700: train_loss = 1.488441944, param_norm = 89.763, grad_norm = 31.863, train_F1/valid_F1 = 0.539000/0.340681
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-4700
27.97 epochs, iter 4750: train_loss = 0.960404217, param_norm = 89.782, grad_norm = 0.144, train_F1/valid_F1 = 0.549000/0.319639
28.27 epochs, iter 4800: train_loss = 0.960650146, param_norm = 89.761, grad_norm = 0.029, train_F1/valid_F1 = 0.533000/0.294589
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-4800
28.56 epochs, iter 4850: train_loss = 0.947577417, param_norm = 89.907, grad_norm = 0.087, train_F1/valid_F1 = 0.530000/0.291583
28.86 epochs, iter 4900: train_loss = 0.957210958, param_norm = 89.900, grad_norm = 0.094, train_F1/valid_F1 = 0.529000/0.304609
Saved parameters to ./all_features_10_seq_100hi

41.52 epochs, iter 7050: train_loss = 0.951350152, param_norm = 92.911, grad_norm = 0.172, train_F1/valid_F1 = 0.540000/0.338677
41.81 epochs, iter 7100: train_loss = 0.952075064, param_norm = 92.960, grad_norm = 0.210, train_F1/valid_F1 = 0.525000/0.276553
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-7100
42.11 epochs, iter 7150: train_loss = 0.955510139, param_norm = 93.140, grad_norm = 0.068, train_F1/valid_F1 = 0.534000/0.277555
42.40 epochs, iter 7200: train_loss = 0.969266832, param_norm = 93.110, grad_norm = 0.415, train_F1/valid_F1 = 0.542000/0.252505
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-7200
42.69 epochs, iter 7250: train_loss = 0.972505748, param_norm = 93.133, grad_norm = 1.572, train_F1/valid_F1 = 0.550000/0.293587
42.99 epochs, iter 7300: train_loss = 0.940819860, param_norm = 93.224, grad_norm = 0.057, train_F1/valid_F1 = 0.536000/0.268537
Saved parameters to ./all_features_10_seq_100hid

55.65 epochs, iter 9450: train_loss = 0.939069092, param_norm = 96.735, grad_norm = 0.084, train_F1/valid_F1 = 0.546000/0.262525
55.94 epochs, iter 9500: train_loss = 0.935321510, param_norm = 96.795, grad_norm = 0.216, train_F1/valid_F1 = 0.559000/0.259519
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-9500
56.24 epochs, iter 9550: train_loss = 0.943100929, param_norm = 96.941, grad_norm = 0.083, train_F1/valid_F1 = 0.554000/0.256513
56.53 epochs, iter 9600: train_loss = 0.953681648, param_norm = 97.003, grad_norm = 0.089, train_F1/valid_F1 = 0.545000/0.249499
Saved parameters to ./all_features_10_seq_100hidden_2layer_drop08_longer/model.ckpt-9600
56.83 epochs, iter 9650: train_loss = 0.953324437, param_norm = 97.157, grad_norm = 0.237, train_F1/valid_F1 = 0.542000/0.261523
57.12 epochs, iter 9700: train_loss = 0.930218399, param_norm = 97.345, grad_norm = 0.060, train_F1/valid_F1 = 0.546000/0.262525
Saved parameters to ./all_features_10_seq_100hid

KeyboardInterrupt: 

In [73]:
# load best checkpoint (based on dev f1) and evaluate
ckpt = tf.train.get_checkpoint_state(experiment_name+'/best_ckpt')
v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
    lstm_model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    raise ValueError('What? you dont have a best checkpoint?')

y_1000_train_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_1000_train, lstm_model.keep_prob: 0.8})
print("train F1:",F1(y_1000_train, y_1000_train_pred))
y_valid_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_valid, lstm_model.keep_prob: 0.8})
print("dev F1:",F1(y_valid, y_valid_pred))
y_test_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_test, lstm_model.keep_prob: 0.8})
print("test F1:",F1(y_test, y_test_pred))

INFO:tensorflow:Restoring parameters from all_features_10_seq_100hidden_2layer_drop08_longer/best_ckpt/best.ckpt-700
train F1: 0.532
dev F1: 0.42084168336673344
test F1: 0.3687374749498998
